# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [3]:
! pip install opencv-python ultralytics

In [40]:
import cv2
import numpy as np
from ultralytics import YOLO

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [41]:
model = YOLO('/content/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [42]:
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [43]:
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
hieght = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# print the fps, width, hieght
print(f'fps : {fps}, width : {width}, hieght : {hieght}')

fps : 29.97002997002997, width : 1920, hieght : 1080


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [44]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output_YOLO_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, hieght))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [45]:
# ROIs
# Covering the Top of the street, (0,0) : this is the top-left corner, (width, height // 4) : this is the bottom-right.
top = (0, 0, width, hieght // 4)

# Covering the bottom of the street, (0, height * 3 // 4) : this is the top-left corner, (width, height) : this is the bottom-right corner.
bottom = (0, hieght * 3 // 4, width, hieght)

# Covering the left of the street, (0,0) : this is the top-left corner, (width // 4, height) : this is the bottom-right corner.
left = (0, 0, width // 4, hieght)

# Covering the right of the street, (width * 3 // 4, 0) : this is the top-left corner, (width, height) : this si the bottom-right corner.
right = (width * 3 // 4, 0, width, hieght)


In [46]:
# this is function for ROIs
def draw_rois(frame):
  # Top
  cv2.rectangle(frame, (top[0], top[1]), (top[2], top[3]), (0,0,255), 1) # the color here Red, thick is 1
  # Bottom
  cv2.rectangle(frame, (bottom[0],bottom[1]), (bottom[2], bottom[3]), (0,255,0), 1) # the color here is Green, thick is 1
  # Left
  cv2.rectangle(frame, (left[0], left[1], left[2], left[3]), (255,0,0), 1) # the coloe here is Blue, thick is 1
  # Right
  cv2.rectangle(frame, (right[0], right[1], right[2], right[3]), (255,255,0), 1) # the color here is light Blue , thick is 1

 Details in ROIs function :
 * here in this cell the (x1,y1) is the top-left corner
 * and the (x2, y2) is the bottom-right corner
 * first am define a 4 variable for the coordenates (top, bottom, left, right)

 * then define a function draw_rois;
    * in the first two parameters its the (x1,y1), and in the last two paramerters its (x2, y2) and the same in avery cv2.rectangle() function.
    * choose a color for each rectangle, then chose the pexels for the rectangle thick.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [47]:
# create variable for Top, bottom, Left, Right
Top_count = 0
Bottom_count = 0
Left_count = 0
Right_count = 0

In [56]:
while True:
    ret,frame = cap.read()
    if not ret:
        break
    draw_rois(frame)

    # YOLO Model :
    results = model(frame)
    # Precces the Detection :
    for result in results:
      for detection in result.boxes:
        x1, y1, x2, y2 = map(int, detection.xyxy[0])
        conf = detection.conf[0] # this is the conf score
        class_id = int(detection.cls[0]) # this is the class ID
        # Bounding Box
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(frame, f'Id : {class_id}, Conf : {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        # Calculated the center of the bounding box
        center_x =(x1 + x2)//2
        center_y =(y1 + y2)//2
        cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255))

        # Create a list to call the variable easer
        exits = [
             (top, 'top'),
             (bottom, 'bottom'),
             (left, 'left'),
             (right, 'right')
             ]

       # count list based on whether it is inside or outside the center
        counts = [direction for exit_area, direction in exits
                    if exit_area[0] <= center_x <= exit_area[2] and exit_area[1] <= center_y <= exit_area[3]]

       # Increment the count based on  match direction
        for direction in counts:
            if direction == 'top':
              Top_count += 1
            elif direction == 'bottom':
              Bottom_count += 1
            elif direction == 'left':
              Left_count += 1
            elif direction == 'right':
              Right_count += 1

      # Display vehicle counts
      cv2.putText(frame, f'Jomanah Saad', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2) # The video should display your name on the top.
      cv2.putText(frame, f'Top Count : {Top_count}', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
      cv2.putText(frame, f'Bottom Count : {Bottom_count}', (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
      cv2.putText(frame, f'Left Count : {Left_count}', (10,90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
      cv2.putText(frame, f'Right Count : {Right_count}', (10,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

      out.write(frame)



0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2997.0ms
Speed: 4.2ms preprocess, 2997.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2949.5ms
Speed: 4.2ms preprocess, 2949.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3466.2ms
Speed: 3.9ms preprocess, 3466.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3557.5ms
Speed: 5.2ms preprocess, 3557.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2762.8ms
Speed: 4.0ms preprocess, 2762.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2790.8ms
Speed: 5.5ms preprocess, 2790.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 3044.8ms
Speed: 4.1ms p

In [57]:
# resources
cap.release()
out.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video

* this the link for video :

https://drive.google.com/file/d/1-4prKUDlGuCx88F1HtBeAFZvYEHhHlz5/view?usp=drive_link